## 12일차 과제 수행 - 김명석

In [1]:
# 오늘 과제 SQLlist 로 넘기는 코드 짜는것 
# URL 집어넣는것 함수형태로 만들어 바꿀 수 있게.


In [2]:
# 측정소별 실시간 측정정보 조회
import urllib , requests
"""
http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty?
serviceKey=3djrRPbmWpPwtLqOCfMXIY0UcD3ZwJXVgX76GuDcdkBLH0VmwvKeoU7IEEZzYoCu6RSeJ4Nq86VoA2%2BUm2iZgw%3D%3D
&numOfRows=10
&pageSize=10
&pageNo=1
&startPage=1
&stationName=%EC%A2%85%EB%A1%9C%EA%B5%AC
&dataTerm=DAILY&ver=1.3
"""
url = "http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"
def_params = {
    "serviceKey":"3djrRPbmWpPwtLqOCfMXIY0UcD3ZwJXVgX76GuDcdkBLH0VmwvKeoU7IEEZzYoCu6RSeJ4Nq86VoA2%2BUm2iZgw%3D%3D",
    "numOfRows":10,
    "pageSize":10,
    "pageNo":1,
    "startPage":1,
    "stationName":None,
    "dataTerm":"DAILY",
    "ver":1.3,
    "_returnType" : "json"
}
def_params["stationName"] = "종로구"
def_params["serviceKey"]  = urllib.parse.unquote(def_params["serviceKey"] )

resp = requests.get(url,params=def_params)

In [9]:
def getResponse(pageNo=1):
    url = "http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"
    def_params = {
        "serviceKey":"3djrRPbmWpPwtLqOCfMXIY0UcD3ZwJXVgX76GuDcdkBLH0VmwvKeoU7IEEZzYoCu6RSeJ4Nq86VoA2%2BUm2iZgw%3D%3D",
        "numOfRows":10,
        "pageSize":10,
        "pageNo":1,
        "startPage":1,
        "stationName":None,
        "dataTerm":"DAILY",
        "ver":1.3,
        "_returnType" : "json"
    }
    def_params["stationName"] = "종로구"
    def_params["serviceKey"]  = urllib.parse.unquote(def_params["serviceKey"] )
    def_params["pageNo"]=1
    resp = requests.get(url,params=def_params)
    return resp 



In [15]:
import sqlite3 as sql
# "MsrstnAcctoRltmMesureDnsty.db"
conn = sql.connect("ArpltnInforInqireSvc.db") 
cur = conn.cursor()

def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


In [4]:
cur.executescript("""
    DROP TABLE IF EXISTS MsrstnAcctoRltmMesureDnsty;

    CREATE TABLE MsrstnAcctoRltmMesureDnsty(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    coGrade INTEGER,
    coValue FLOAT,
    dataTerm INTEGER,
    dataTime TEXT,
    khaiGrade INTEGER,
    khaiValue INTEGER,
    mangName TEXT,
    no2Grade INTEGER,
    no2Value FLOAT,
    numOfRows INTEGER,
    o3Grade INTEGER,
    o3Value FLOAT,
    pageNo INTEGER,
    pm10Grade INTEGER,
    pm10Grade1h INTEGER,
    pm10Value INTEGER,
    pm10Value24 INTEGER,
    pm25Grade INTEGER,
    pm25Grade1h INTEGER,
    pm25Value INTEGER,
    pm25Value24 INTEGER,
    resultCode INTEGER,
    resultMsg TEXT,
    rnum INTEGER,
    serviceKey TEXT,
    sidoName TEXT,
    so2Grade INTEGER,
    so2Value FLOAT,
    stationCode TEXT,
    stationName TEXT,
    totalCount INTEGER,
    ver FLOAT);""")

In [12]:
import json

def insertRespToDB(resp):
    respObj = json.loads(resp.text)
    name = ""
    value= ""
    qurey = ""
    for list_dic in respObj['list']:
        for k,v in list_dic.items():
                name = name + k +', '
                if str(v) is not '':
                    if str(v).isnumeric() or isfloat(str(v)):
                        value= value + str(v) + ', '
                    else :
                        value= value + '\''+str(v) + '\', '
                else :
                    value= value + 'NULL' + ', '
        name = name[13:-2]
        value = value[7:-2]
        qurey = "insert into MsrstnAcctoRltmMesureDnsty (" + name + ") values (" +value +");"

        cur.execute(qurey)
        name = ""
        value= ""
        qurey = ""
     

In [17]:
for i in range(2,10):
    resp = getResponse(pageNo=i)
    insertRespToDB(resp)
conn.commit()

In [18]:
for i in cur.execute("select *from MsrstnAcctoRltmMesureDnsty").fetchall():
    print(i)

(1, 1, 0.4, None, '2018-07-17 20:00', 2, 79, '도시대기', 1, 0.024, 10, 1, 0.013, 1, 2, 1, 28, 32, 2, 2, 27, 27, None, None, 0, None, None, 1, 0.004, None, None, None, None)
(2, 1, 0.3, None, '2018-07-17 19:00', 2, 79, '도시대기', 1, 0.017, 10, 1, 0.02, 1, 2, 1, 27, 34, 2, 2, 22, 27, None, None, 0, None, None, 1, 0.004, None, None, None, None)
(3, 1, 0.3, None, '2018-07-17 18:00', 2, 82, '도시대기', 1, 0.014, 10, 1, 0.019, 1, 2, 1, 27, 35, 2, 2, 25, 28, None, None, 0, None, None, 1, 0.004, None, None, None, None)
(4, 1, 0.2, None, '2018-07-17 17:00', 2, 82, '도시대기', 1, 0.014, 10, 1, 0.02, 1, 2, 2, 37, 37, 2, 2, 34, 28, None, None, 0, None, None, 1, 0.006, None, None, None, None)
(5, 1, 0.3, None, '2018-07-17 16:00', 2, 79, '도시대기', 1, 0.017, 10, 1, 0.028, 1, 2, 2, 38, 37, 2, 3, 36, 27, None, None, 0, None, None, 1, 0.008, None, None, None, None)
(6, 1, 0.3, None, '2018-07-17 15:00', 2, 77, '도시대기', 1, 0.019, 10, 2, 0.036, 1, 2, 2, 42, 36, 2, 3, 37, 26, None, None, 0, None, None, 1, 0.008, None, None, 